# Importing Libraries

In [4]:
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# Load preprocessed data

In [2]:
X = np.load('data/preprocessed/features.npy')
Y = np.load('data/preprocessed/labels.npy')

# Split the data into training and testing set

In [3]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.4, random_state=42)

# Normalize the data

In [8]:
x_train = x_train/x_train.max()
x_test = x_test/x_test.max()

# One hot encoding the labels

In [32]:
y_train

array([0, 6, 9, ..., 9, 9, 6])

In [26]:
def one_hot_encode(arr:np.array):
    a = arr.max()
    one_hot = np.zeros((len(arr), a))
    final = []
    for i,j in zip(arr, one_hot):
        one_hot_row = np.insert(j, i, 1)
        final.append(one_hot_row)
    one_hot = np.array(final)
    return(one_hot)


In [33]:
y_train = y_train.astype(int)
y_test = y_test.astype(int)

In [34]:
y_train = one_hot_encode(y_train)
y_test = one_hot_encode(y_test)